In [4]:
# import libraries

import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [3]:
#Define Harvesine distance foormula 
#input are coordinates of consecutive points in the route lon1, lat1, lon2, lat2
# retuns the distance between them

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometer between two points on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    
    c = 2 * asin(sqrt(a)) 
    r = 6371# radius of earth in kilometers is 6371
    km = r* c
    return km

In [5]:
#Read dat using pandas library from gps.csv file
#This is phyphox gps data from the phone which i took while walking short distance in home surroundings

data = pd.read_csv('gps.csv')
data.head()

,Time (s),Latitude (°),Longitude (°),Altitude (m),Altitude WGS84 (m),Speed (m/s),Direction (°),Distance (km),Horizontal Accuracy (m),Vertical Accuracy (m),Satellites
0,0.000000,65.012458,25.504513,NaN,NaN,NaN,NaN,0.232095,1299.999023,0.000000,0.0
1,8.147709,65.011114,25.500732,55.274993,72.739014,0.54,0.000000,0.232915,5.193503,8.638241,0.0
2,8.985769,65.011107,25.500739,53.185517,70.649536,0.48,0.000000,0.236528,6.715902,10.220663,16.0
3,9.302697,65.011080,25.500781,40.583349,58.047363,1.11,358.600006,0.237158,4.208384,8.488465,16.0
4,9.318649,65.011086,25.500781,40.546482,58.010498,1.11,358.600006,0.240923,4.405093,8.488465,16.0


In [9]:
#dervie velocity using haversine distance and time difference between consecutive points (although it is already given in the data)

lat = data['Latitude (°)'] #latitude
lon = data['Longitude (°)'] #longitude

#defining two empty coloums for distance and time difference
data['dist'] = np.zeros(lat.shape[0]) # Add new coloum of Zeroes to the data. we will fill it with distance values
data['time_diff'] = np.zeros(lat.shape[0]) # Add new coloum of Zeroes to the data. we will fill it with time difference values

#loop over all the points in the data and calculate distance and time difference between consecutive points. for loop goes until 
#the second last point
for i in range(lat.shape[0]-1):
    data['dist'][i] = haversine(lon[i], lat[i], lon[i+1], lat[i+1])*1000 #calculate distance between data points with indices i and i+1 in meters
    data['time_diff'][i] = data['Time (s)'][i+1] - data['Time (s)'][i] #calculate time difference/ time step between data points with indices i and i+1 in seconds

#calculate velocity using distance and time difference
data['dist'][0] = 0 #set distance of first point to zero    
data['velocity'] = data['dist']/data['time_diff'] #velocity in m/s

data.head(10)

,Time (s),Latitude (°),Longitude (°),Altitude (m),Altitude WGS84 (m),Speed (m/s),Direction (°),Distance (km),Horizontal Accuracy (m),Vertical Accuracy (m),Satellites,dist,time_diff,velocity
0,0.000000,65.012458,25.504513,NaN,NaN,NaN,NaN,0.232095,1299.999023,0.000000,0.0,0.000000,8.147709,0.000000
1,8.147709,65.011114,25.500732,55.274993,72.739014,0.54,0.000000,0.232915,5.193503,8.638241,0.0,0.820436,0.838060,0.978970
2,8.985769,65.011107,25.500739,53.185517,70.649536,0.48,0.000000,0.236528,6.715902,10.220663,16.0,3.612921,0.316928,11.399820
3,9.302697,65.011080,25.500781,40.583349,58.047363,1.11,358.600006,0.237158,4.208384,8.488465,16.0,0.629554,0.015952,39.464809
4,9.318649,65.011086,25.500781,40.546482,58.010498,1.11,358.600006,0.240923,4.405093,8.488465,16.0,3.765073,1.023771,3.677652
5,10.342420,65.011054,25.500751,31.046619,48.510620,1.10,347.200012,0.241922,4.301970,7.521406,32.0,0.999776,1.000396,0.999380
6,11.342817,65.011050,25.500732,26.474662,43.938660,0.95,349.399994,0.242366,3.790092,5.113861,26.0,0.444072,0.992687,0.447344
7,12.335503,65.011052,25.500723,24.759636,42.223633,0.83,348.100006,0.243043,3.790092,4.295459,30.0,0.676672,1.008617,0.670891
8,13.344120,65.011058,25.500723,23.603140,41.067139,1.29,359.100006,0.244141,3.790092,3.497247,30.0,1.097797,0.996904,1.101206
9,14.341024,65.011067,25.500715,22.899951,40.363953,1.16,348.799988,0.245333,3.790092,3.184589,30.0,1.191641,0.994283,1.198493


In [17]:
import folium

location_frame = data[['Latitude (°)','Longitude (°)']] #create data frame with mean latitude and longitude of the data
location_list = location_frame.values.tolist() #convert data frame to list

avg_lat = np.mean(location_list, axis = 0)[0] #calculate average latitude
avg_lon = np.mean(location_list, axis = 0)[1] #calculate average longitude

#Load map centred on average coordinates
my_map = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

#add a markers to the map for each point in the data
# for each in location_list:
#     folium.Marker(each).add_to(my_map)

#Draw lines of motiion 
folium.PolyLine(location_list, color="red", weight=2.5, opacity=1).add_to(my_map)

#save the map as html file
my_map.save("Sanjay_map.html")

#Show the map again
my_map



